# Connect 4

---

Author: S. Menary [sbmenary@gmail.com]

Date  : 2023-01-18, last edit 2023-01-18

Brief : Play a tournment between two bots to determine which is stronger

---

## Imports

In [1]:
##=====================================##
##  All imports should be placed here  ##
##=====================================##

##  Python core libs
import sys, time

##  PyPI libs
import numpy as np

##  Local packages
from connect4.utils    import DebugLevel
from connect4.game     import BinaryPlayer, GameBoard, GameResult
from connect4.MCTS     import PolicyStrategy
from connect4.bot      import Bot_NeuralMCTS, Bot_VanillaMCTS
from connect4.parallel import MonitorThread, WorkerThread, kill_threads


In [2]:
##=====================================##
##  Print version for reproducibility  ##
##=====================================##

print(f"Python version is {sys.version}")
print(f"Numpy  version is {np.__version__}")


Python version is 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Numpy  version is 1.23.2


##  MCTS

The `Bot_NeuralMCTS` object is used to apply bot actions using MCTS with a loaded model.

In [3]:
##=========================================##
##  Define methods for playing tournament  ##
##=========================================##


def play_bot_game(model1, model2, duration:int=1, discount=1., debug_lvl:DebugLevel=DebugLevel.MUTE) :
    ##  Create game and bots
    policy_strategy = PolicyStrategy.SAMPLE_POSTERIOR_POLICY
    game_board = GameBoard()
    bot1       = Bot_NeuralMCTS(model1, policy_strategy=policy_strategy) if model1 else Bot_VanillaMCTS(policy_strategy=policy_strategy)
    bot2       = Bot_NeuralMCTS(model2, policy_strategy=policy_strategy) if model2 else Bot_VanillaMCTS(policy_strategy=policy_strategy)
    debug_lvl.message(DebugLevel.LOW, f"Using bot1 {bot1}")
    debug_lvl.message(DebugLevel.LOW, f"Using bot2 {bot2}")
    debug_lvl.message(DebugLevel.LOW, game_board)
    
    ##  Shuffle bots
    ret = {"model1":BinaryPlayer.X, "model2":BinaryPlayer.O}
    if np.random.choice([True, False]) :
        bot1, bot2 = bot2, bot1
        ret = {"model1":BinaryPlayer.O, "model2":BinaryPlayer.X}

    ##  Take moves until end of game
    result = game_board.get_result()
    while not result :
        bot = bot1 if game_board.to_play == BinaryPlayer.X else bot2
        bot.take_move(game_board, duration=duration, discount=discount, debug_lvl=debug_lvl)
        result = game_board.get_result()
        
    ##  Return
    ret["result"] = result
    return ret


def bot_tournament_process(proc_idx, num_games, out_queue, base_seed, model1_name, model2_name, duration, discount) :
    from connect4.neural import load_model
    np.random.seed(base_seed+proc_idx)
    model1 = load_model(model1_name) if len(model1_name) > 0 else None
    model2 = load_model(model2_name) if len(model2_name) > 0 else None
    for game_idx in range(num_games) :
        _ = play_bot_game(model1, model2, duration, discount)
        out_queue.put(_)
    
    
def generate_datapoints(num_processes, num_games_per_proc, base_seed, model1_name, model2_name, 
                        duration, discount, mon_freq=3) :
    worker  = WorkerThread(bot_tournament_process, num_processes, num_games_per_proc, 
                       func_args=[base_seed, model1_name, model2_name, duration, discount])
    monitor = MonitorThread(worker, frequency=mon_freq)

    monitor.start()
    worker .start()

    worker .join()
    monitor.join()
    
    return worker.results


In [4]:
##================##
##  Load a model  ##
##================##

model_name_old = "../models/.neural_model_v2.h5"
model_name_new = "../models/.neural_model_v3.h5"


In [5]:
##===================================##
##  Run processes to obtain results  ##
##===================================##

results = generate_datapoints(7, 10, 10, model_name_old, model_name_new, 3, .99, mon_freq=3)


Generated 70 / 70 results [t=953.35s]
Generation complete [t=953.36s] [n=70]


In [8]:
##========================##
##  Report on the scores  ##
##========================##

score = [[0, 0, 0, 0], [0, 0, 0, 0]]

for result in results :
    score_idx = 0
    score_idx = 0 if result["model1"] == BinaryPlayer.X else 1
    result_idx = 0
    if result["result"] == GameResult.X    : result_idx = 1
    if result["result"] == GameResult.O    : result_idx = 2
    if result["result"] == GameResult.DRAW : result_idx = 3
    score[score_idx][result_idx] += 1
    
print(f"Results (LHS player goes first): Old Model {score[0][1]} - {score[0][2]} New Model")
print(f"Results (LHS player goes first): New Model {score[1][1]} - {score[1][2]} Old Model")
    

Results (LHS player goes first): Old Model 9 - 29 New Model
Results (LHS player goes first): New Model 21 - 8 Old Model
